In [1]:
import os
import sys

import numpy as np
import pandas as pd
from sqlalchemy import create_engine


sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('../..'))))
from strategy import EquityStrategy
from tester import Tester


base_path = os.path.abspath('../../..')
data_path = os.path.join(base_path, 'data')
database_path = os.path.join(data_path, 'database')
strategy_path = os.path.join(base_path, 'strategy')
check_path = os.path.join(strategy_path, 'check')
sys.path.append(strategy_path)

In [2]:
from strategy import CommodityStrategy, EquityStrategy, IRStrategy, EmergingStrategy
from tester import Tester

In [3]:
eindex_path = os.path.join(data_path, 'totindex.csv')
eindex1_path = os.path.join(data_path, 'priceindex.csv')
efuture_path = os.path.join(data_path, 'FutGenratio1.csv')

In [4]:
eindex_path = os.path.join(data_path, 'totindex.csv')
eindex1_path = os.path.join(data_path, 'priceindex.csv')
efuture_path = os.path.join(data_path, 'FutGenratio1.csv')

eindex = pd.read_csv(eindex_path, header=0, index_col=0, parse_dates=True)
eindex1 = pd.read_csv(eindex1_path, header=0, index_col=0, parse_dates=True)
efut = pd.read_csv(efuture_path, header=0, index_col=0, parse_dates=True)

eindex1 = eindex1.loc[:'2008']
ERetp = eindex1.pct_change(1).iloc[1:]
ERett = eindex.pct_change(1).iloc[1:]
fRet = efut.pct_change(1).iloc[1:]

compRet = ERett
compRet[compRet.isna()] = ERetp[compRet.isna()]
ERet1 = compRet
ERet1.drop_duplicates(inplace=True)
fRet.drop_duplicates(inplace=True)
ERet = pd.concat([ERet1.loc[:'2007-12-31'], fRet.loc['2008-01-01':]], axis=0)
ERet = ERet.fillna(0)
Eindex = (1 + ERet).cumprod()
Eindex = Eindex / Eindex.iloc[0]

In [5]:
class EQL(EquityStrategy):
    def __init__(self, strategy_name, asset_type):
        super().__init__(strategy_name=strategy_name, asset_type=asset_type)
        self.eps = None

    def load_strategy_data(self, table='factset', origin1='ROA', origin2="ICR"):
        self.roa = self._load_strategy_data(table=table, origin=origin1)
        self.icr = self._load_strategy_data(table=table, origin=origin2)

    def calculate_signal(self, minobs1=60, nopos=0.4, CS=0.35, lag=1):
        """

        :param cs_num: percentage of position for cross sectional signal
        :param min_obs:
        :param longlen: long term price momentum period
        :param shortlen: short term price momentum period
        :return:
        """
        self.logger.info('[STEP 3] CACULATE SIGNAL')
        for i in range(2):
            if i == 0:
                RET = self.ret.loc[:, ['SPX', 'TSX', 'FTSE', 'DAX', 'CAC', 'SMI', 'MIB', 'IBEX', 'OMX', 'AEX']]
            else:
                RET = self.ret.loc[:, ['NKY', 'AS51', 'HSI', "SG"]]

            index = self.index.loc[:, RET.columns]

            # 1. Load Data
            roa = self.roa.copy()
            roa = roa[index.columns]

            icr = self.icr.copy()
            icr = icr[index.columns]

            # 3. RV
            RV1 = roa
            RV = (RV1 - RV1.shift(lag)).iloc[lag:]

            # 4. rank
            pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
            RVrank = RV.expanding().apply(pctrank, raw=True)  # it takes some time
            RVrank = RVrank.iloc[minobs1 - 1:, ]

            # 5. trade
            truecount = (RVrank.notnull().sum(axis=1) * CS).apply(round)
            tiebreaker = RVrank.rolling(5).mean().fillna(0) * 0.0000001

            # 1. Cross sectional
            CSRV = (RVrank + tiebreaker).rank(axis=1, method='first')  # Short
            CSRV1 = (-1 * RVrank - 1 * tiebreaker).rank(axis=1, method='first')  # Long

            CSRVpos = CSRV * 0
            CSRVpos[CSRV.apply(lambda x: x <= truecount, axis=0)] = -1
            CSRVpos[CSRV1.apply(lambda x: x <= truecount, axis=0)] = 1

            CSRV = CSRVpos
            CSRV.fillna(0, inplace=True)
            CSRVone = CSRV

            # 2. Time Series
            TSRV1 = RVrank * 0
            TSRV1[RVrank > nopos + (1 - nopos) / 2] = 1  # Long
            TSRV1[RVrank < (1 - nopos) / 2] = -1  # Short

            # 2. Signal2
            RV1 = icr
            RV = (RV1 - RV1.shift(lag)).iloc[lag:]

            # 1. rank
            pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
            RVrank = RV.expanding().apply(pctrank, raw=True)  # it takes some time
            RVrank = RVrank.iloc[minobs1 - 1:, ]

            # 2. trade
            truecount = (RVrank.notnull().sum(axis=1) * CS).apply(round)
            tiebreaker = RVrank.rolling(5).mean().fillna(0) * 0.0000001

            # 1. Cross sectional
            CSRV = (RVrank + tiebreaker).rank(axis=1, method='first')  # Short
            CSRV1 = (-1 * RVrank - 1 * tiebreaker).rank(axis=1, method='first')  # Long

            CSRVpos = CSRV * 0
            CSRVpos[CSRV.apply(lambda x: x <= truecount, axis=0)] = -1
            CSRVpos[CSRV1.apply(lambda x: x <= truecount, axis=0)] = 1

            CSRV = CSRVpos
            CSRV.fillna(0, inplace=True)
            CSRVtwo = CSRV

            # 2. Time Series
            TSRV2 = RVrank.fillna(0) * 0
            TSRV2[RVrank > nopos + (1 - nopos) / 2] = 1  # Long
            TSRV2[RVrank < (1 - nopos) / 2] = -1  # Short

            TSRV = (2 * TSRV1 + TSRV2) / 3
            CSRV = (CSRVone + 2 * CSRVtwo) / 3

            if i == 0:
                TSRVrun1 = TSRV
                CSRVrun1 = CSRV
            else:
                TSRVrun2 = TSRV
                CSRVrun2 = CSRV

        TSRV = pd.concat([TSRVrun1, TSRVrun2], axis=1)
        CSRV = pd.concat([CSRVrun1, CSRVrun2], axis=1)

        TSRV = TSRV[self.ret.columns]
        CSRV = CSRV[self.ret.columns]

        self.TSRV = TSRV.loc[self.ret.index].fillna(method='ffill').dropna(how='all')
        self.CSRV = CSRV.loc[self.ret.index].fillna(method='ffill').dropna(how='all')

        # Align dates with each other
        if self.TSRV.index[0] > self.CSRV.index[0]:
            self.CSRV = self.CSRV.loc[self.TSRV.index[0]:]
        else:
            self.TSRV = self.TSRV.loc[self.CSRV.index[0]:]

In [85]:
eql = EQL(strategy_name="EQL", asset_type="EQUITY")
eql.index = Eindex.copy()
eql.ret = ERet.copy()

# eql.load_index_and_return(from_db=False, save_file=False)
# eql.load_strategy_data(table='FS', origin1='ROA', origin2='ICR')
roa = pd.read_csv(os.path.join(data_path, "ROA.csv"), index_col=0, parse_dates=True)
roa.index.name = 'tdate'
roa.columns.name = 'ticker'
eql.roa = roa

icr = pd.read_csv(os.path.join(data_path, "ICR.csv"), index_col=0, parse_dates=True)
icr.index.name = 'tdate'
icr.columns.name = 'ticker'
eql.icr = icr

eql.set_rebalance_period(freq='month')  # rebalance_day: monday = 0, sunday = 6
eql.calculate_signal(minobs1=12, nopos=0.4, CS=0.35, lag=1)

2019-12-20 15:16:43,240 - EQL - INFO - [STEP 0] START LOGGING EQL
2019-12-20 15:16:43,248 - EQL - INFO - [STEP 2] SET REBALANCE PERIOD
2019-12-20 15:16:43,249 - EQL - INFO - [STEP 3] CACULATE SIGNAL
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:111: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:112: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike


In [87]:
eql.TSRV.to_csv('../past/eql_tsrv_python.csv')

In [88]:
eql.CSRV.to_csv('../past/eql_csrv_python.csv')

In [98]:
eql_TSRV = pd.read_csv('../past/eql_tsrv.csv', index_col=[0])
eql_CSRV = pd.read_csv('../past/eql_csrv.csv', index_col=[0])

In [101]:
eql_TSRV.index = pd.to_datetime(eql_TSRV.index)
eql_CSRV.index = pd.to_datetime(eql_CSRV.index)

In [132]:
TSRV_DIFF = (eql_TSRV - eql.TSRV)

In [133]:
TSRV_DIFF[np.abs(TSRV_DIFF) > 0.01].count()

SPX     0
TSX     0
FTSE    0
DAX     0
CAC     0
SMI     0
MIB     0
IBEX    0
OMX     0
AEX     0
NKY     0
AS51    0
HSI     0
SG      0
dtype: int64

In [134]:
TSRV_DIFF.sum(axis=0)

SPX    -9.936496e-15
TSX     1.346701e-13
FTSE    1.331157e-13
DAX     1.542655e-13
CAC     8.587575e-14
SMI     1.713074e-13
MIB     7.044365e-14
IBEX    9.692247e-14
OMX     9.586776e-14
AEX     2.448042e-14
NKY     3.419487e-14
AS51    1.846856e-13
HSI     2.553513e-15
SG     -1.591505e-13
dtype: float64

In [135]:
CSRV_DIFF = (eql_CSRV - eql.CSRV)

In [145]:
CSRV_DIFF[np.abs(CSRV_DIFF) > 0.001].sum(axis=0)

SPX     -7.000000
TSX    -14.666667
FTSE     0.000000
DAX    -14.333333
CAC      0.000000
SMI      0.000000
MIB      7.000000
IBEX     0.000000
OMX     14.666667
AEX     14.333333
NKY      0.000000
AS51     0.000000
HSI      0.000000
SG       0.000000
dtype: float64